In [ ]:
import numpy as np
import cv2 as cv
import math 
import os


# Código para caputar fotos del dataset

In [ ]:

# path = 'riesgoConRuido/incendios/incendio'
# path = 'riesgoConRuido/tornados/tornado'
# path = 'riesgoConRuido/inundaciones/inundacion'
# path = 'riesgoConRuido/asaltos/asalto'
path = 'riesgoConRuido/robos_a_casas/robocasa'

partes = path.split('/')
print(partes)

path_video = '/'.join(partes[-2:])
print(path_video)

acumulado = 2652

cap = cv.VideoCapture(f"videos/{path_video}5.mp4")
fps = cap.get(cv.CAP_PROP_FPS)  # Obtiene la velocidad de fotogramas
frames_to_skip = int(fps * 5)  # Calcula los fotogramas a adelantar (5 segundos)

i = 0

while True:
    ret, frame = cap.read()
    if ret:
        # Muestra el tiempo transcurrido
        current_frame = int(cap.get(cv.CAP_PROP_POS_FRAMES))
        current_time = current_frame / fps
        time_text = f"Time: {current_time:.2f} sec"
        cv.putText(frame, time_text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv.imshow('img', frame)
        k = cv.waitKey(30)  # Aumenta este valor para hacer que el video se reproduzca más lento

        if k == ord('a'):
            i += 1
            cv.imwrite(f'imgs/{path}' + str(i+acumulado) + '.jpg', frame)
        
        if k == ord('f'):  # Adelanta 5 segundos 
            current_frame += frames_to_skip
            if current_frame >= cap.get(cv.CAP_PROP_FRAME_COUNT):
                break
            cap.set(cv.CAP_PROP_POS_FRAMES, current_frame)

        if k == 27:  # tecla ESC para salir
            break
    else:
        break

cap.release()
cv.destroyAllWindows()



# Procesos para modificar imágenes del dataset

In [ ]:
import cv2 as cv
import os


crop_top = 0.1
crop_bottom = 0.06
crop_left = 0.0 
crop_right = 0.0



def crop_image(img, i, file, nameFile):

    """Funcion para recortar el tiempo"""

    height, width = img.shape[:2]
    
    # Calcular las coordenadas de recorte
    start_row = int(crop_top * height)
    end_row = int((1 - crop_bottom) * height)
    start_col = int(crop_left * width)
    end_col = int((1 - crop_right) * width)
    
    cropped_img = img[start_row:end_row, start_col:end_col]

    path = f'imgs/riesgoSinRuido/{file}NR'

    if not os.path.exists(path):
        os.makedirs(path)

    cv.imwrite(f'{path}/{nameFile}'+str(i)+'.png', cropped_img)


def resizeimg(img, i, file, nameFile):
    
    """Funcion para redimensionar imagen a 28x21 px"""

    path = f'imgs/riesgo/{file}Reduce'

    if not os.path.exists(path):
        os.makedirs(path)

    frame2 = cv.resize(img, (28,21), interpolation = cv.INTER_AREA)

    cv.imwrite(f'{path}/{nameFile}reduce'+str(i)+'.png', frame2)


def espejo(img, i, file, nameFile):
    # Crea una imagen espejo usando flip

    path = f'imgs/riesgo/{file}Mirror'

    if not os.path.exists(path):
        os.makedirs(path)

    img_espejo = cv.flip(img, 1)
    # Guarda la imagen espejo
    cv.imwrite(f'{path}/{nameFile}mirror'+str(i)+'.png', img_espejo)


####################### Funciones Globales ###################################

path_general = './imgs/riesgoSinRuido'


## Eliminar Ruido de imagenes

def crop_all_images(file):
    i = 0
    imgPaths = f'./imgs/riesgoConRuido/{file}'
    nomFiles = os.listdir(imgPaths)

    for nomFile in nomFiles:
        i = i+1
        imgPath=imgPaths+"/"+nomFile
        img = cv.imread(imgPath)        
        nameFile = removeNum_and_Type(nomFile)
        crop_image(img, i, file, nameFile)
        nameFile = ''


## Redimensionar imagenes

def resize_all_images(file):

    i = 0
    imgPaths = f'{path_general}/{file}NR'
    nomFiles = os.listdir(imgPaths)
    for nomFile in nomFiles:
        
        i = i+1
        imgPath=imgPaths+"/"+nomFile
        img = cv.imread(imgPath)
        nameFile = removeNum_and_Type(nomFile)
        resizeimg(img, i, file, nameFile) 
        nameFile = ''

## Generar imágenes espejo

def mirror_all_images(file):
    i = 0
    imgPaths = f'./imgs/riesgo/{file}Reduce'
    nomFiles = os.listdir(imgPaths)
    for nomFile in nomFiles:
        i = i+1
        imgPath=imgPaths+"/"+nomFile
        img = cv.imread(imgPath)
        nameFile = removeNum_and_Type(nomFile)
        espejo(img, i, file, nameFile) 
        nameFile = ''




In [ ]:
files  = os.listdir('./imgs/riesgoConRuido/')


for file in files:
        
    # llama funcion para recortar imagenes
    # crop_all_images(file)

    # llama funcion para redimensionar imagenes
    # resize_all_images(file)

    # # llama funcion para obtener espejo imagenes
    mirror_all_images(file)
        



In [6]:
import re


## tomarnombreFile
def removeNum_and_Type(nameFile) -> str:

    pattern = re.compile(r'\d+\.(jpg|png)$')

    return pattern.sub('', nameFile)


# Conversion de png a jpg 

In [8]:
from PIL import Image
import os
import cv2 as cv


path = './imgs/dataset'

def convertImg(imgPath, i, file, nameFile):
    # Abre la imagen .png
    img = Image.open(imgPath)

    # Convierte la imagen a RGB (para evitar problemas con imágenes RGBA)
    img_rgb = img.convert('RGB')

    # Guarda la imagen en formato .jpg
    img_rgb.save(f'{path}/{file}/{nameFile}'+str(i)+'.jpg')


def convert_all_Imgs(file):

    i = 0
    imgPaths = f'{path}/{file}'
    nomFiles = os.listdir(imgPaths)
    for nomFile in nomFiles:
        i = i+1
        imgPath=imgPaths+"/"+nomFile
        nameFile = removeNum_and_Type(nomFile) 
        convertImg(imgPath, i, file, nameFile) 
        nameFile = ''

def delete_file_PNG(file):

    i = 0
    imgPaths = f'{path}/{file}'  
    nomFiles = os.listdir(imgPaths)

    for nomFile in nomFiles:
        if nomFile.endswith('.png'):
            imgPath=imgPaths+"/"+nomFile
            os.remove(imgPath)
            i = i+1

#############################################

files  = os.listdir(path)

for file in files:
        
    ## llama funcion para convertir espejo imagenes
    # convert_all_Imgs(file)

    ## borra todos los *.png
    if os.path.isdir(os.path.join(path, file)):
        delete_file_PNG(file)




In [ ]:


i = 0
imgPaths = 'riesgos/tornados'
nomFiles = os.listdir(imgPaths)
for nomFile in nomFiles:
    i = i+1
    imgPath=imgPaths+"/"+nomFile
    convertImg(imgPath, i)


In [ ]:
import os

# Directorio donde están las imágenes
img_dir = 'riesgos/tornados'

# Lista de todos los archivos en el directorio
files = os.listdir(img_dir)

# Recorre todos los archivos
for file in files:
    # Si el archivo es una imagen .png
    if file.endswith('.png'):
        # Construye la ruta completa del archivo
        file_path = os.path.join(img_dir, file)
        # Elimina el archivo
        os.remove(file_path)


i = 0
imgPaths = 'riesgos/asaltos'
nomFiles = os.listdir(imgPaths)
for nomFile in nomFiles:
    i = i+1
    imgPath=imgPaths+"/"+nomFile
    img = cv.imread(imgPath)
    convert_to_gray(img, i)

In [ ]:
from skimage.transform import resize

images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
#filenames = ['test/asalto6.jpg']
filenames = ['test/incendio.jpg']
# filenames = ['test/inundacion1.jpg']
# filenames = ['test/tornado3.jpg']

resultado = " "
frame = cv.imread(filenames[0])

for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (28, 21),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
test_X = X.astype('float32')
test_X = test_X / 255.

predicted_classes = sport_model.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    print(filenames[i], deportes[img_tagged.tolist().index(max(img_tagged))])
    resultado = deportes[img_tagged.tolist().index(max(img_tagged))]
    
print(resultado)
    
cv.imshow("La situacion de riesgo es:    -"+resultado, frame)
cv.waitKey(0)
cv.destroyAllWindows()